# Named Entity Recognition Pretrained Model
# Examples


In [9]:
import spacy
import pandas as pd
import numpy as np
from spacy import displacy
%load_ext autoreload
%autoreload 2
from data_prep import search_keyword_loc_old,filter_spans,convert_to_spacy

In [2]:


text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.render(doc, style="ent")


In [10]:
df=pd.read_excel('data_masking.xlsx')

<ipython-input-10-02f3202cf741>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df=pd.read_excel('data_masking.xlsx')


In [4]:
df['description'][1]

'[First Amendment effective 01-May-17 to DOW For Compare Test - Org Confirmations effective 12-Sep-16 to RR Agreement effective 01-Jan-12 to SN Agreement effective 01-Jan-12 between ABC and  XYZ, Inc.]'

In [5]:
for i in df['description']:
    text = i
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char,ent.label_)
    displacy.render(doc, style="ent")

12 66 68 CARDINAL
01-Jan-12 102 111 DATE
01-Jan-12 138 147 DATE
ABC 156 159 ORG
XYZ 164 167 ORG
months 241 247 DATE
& Renewal Notes 260 275 ORG


First Amendment 1 16 ORG
01 27 29 CARDINAL
Compare Test - Org Confirmations 48 80 ORG
12 91 93 CARDINAL
01-Jan-12 127 136 DATE
01-Jan-12 163 172 DATE
ABC 181 184 ORG
XYZ, Inc. 190 199 ORG


31 69 71 CARDINAL
01-Jan-26 105 114 TIME
01-Jan-12 141 150 DATE
ABC 159 162 ORG
XYZ 168 171 ORG
External Party Current Name 219 246 ORG
0069082-XYZ 248 259 PERSON
90];[Notice 310 321 CARDINAL
months 381 387 DATE
& Renewal Notes 400 415 ORG


Tenth Amendment 1 16 LAW
01 27 29 CARDINAL
31 85 87 CARDINAL
01-Jan-12 121 130 DATE
01-Jan-12 157 166 DATE
ABC 175 178 ORG
XYZ, Inc. 184 193 ORG


Tenth Amendment 1 16 LAW
07-Jul-16 27 36 DATE
31 85 87 CARDINAL
01-Jan-12 121 130 DATE
01-Jan-12 157 166 DATE
ABC 175 178 ORG
XYZ, Inc. 183 192 ORG


BT Application Management Services 32 66 ORG
01 77 79 CARDINAL
01-Jun-13 120 129 DATE
ABC 138 141 ORG
XYZ Inc.]];[Contract No 146 169 ORG
External Party Current Name 196 223 ORG
90];[Notice 284 295 CARDINAL
months 355 361 DATE
& Renewal Notes 374 389 ORG


100279 10 16 DATE
01 46 48 CARDINAL
BCBT Application Management Services 67 103 ORG
01 114 116 CARDINAL
01-Jun-13 157 166 DATE
ABC 175 178 ORG
XYZ Inc. 183 191 ORG
1 212 213 CARDINAL
1 year 235 241 DATE


01-Dec-20 27 36 DATE
BCBT Application Management Services 48 84 ORG
01 95 97 CARDINAL
01-Jun-13 138 147 DATE
ABC 156 159 ORG
XYZ Inc. 164 172 ORG


01 45 47 CARDINAL
ABC 63 66 ORG
XYZ (Canada 71 82 ORG
TAT-32998];[External Party Current Name 108 147 ORG
30];[Notice 212 223 CARDINAL
months 285 291 DATE
& Renewal Notes 303 318 ORG


Data Integrity Services 23 46 ORG
03-Oct-16 57 66 DATE
01-Jan-12 93 102 DATE
01-Jan-12 129 138 DATE
ABC 147 150 ORG
XYZ 156 159 ORG
months 234 240 DATE
& Renewal Notes 253 268 ORG


# Data Preprocessing For text input

In [2]:

import pandas as pd
from tqdm.notebook import tqdm
import spacy
from spacy.tokens import DocBin

def search_keyword_loc_old(texts,dict_keywords_label):
    '''
    This Function is used to:
    Locate the keyword we want
    dict_indext: The dictionary including the keyword and their corresponding label
    return: the list with input text data with location of keywords(start and end position)
    '''
    list_res=[]
    for text in texts:
        dict_index={"entity":list()}
        for i in range(len(dict_keywords_label)):

            for key, label in dict_keywords_label[i].items():
                if text.find(key)!=-1:
                    dict_index["entity"].append((text.find(key),
                                                    (len(key)+text.find(key)),   label))

        list_res.append((text,dict_index))
    return list_res



def filter_spans(spans):
#  """Filter a sequence of spans and remove duplicates or overlaps. Useful for
#  creating named entities (where one token can only be part of one entity) or
#  when merging spans with `Retokenizer.merge`. When spans overlap, the (first)
#  longest span is preferred over shorter spans.

#  spans (iterable): The spans to filter.
#  RETURNS (list): The filtered spans.
#  """
    get_sort_key = lambda span: (span.end - span.start, -span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
     # Check for end - 1 here because boundaries are inclusive
     if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    result = sorted(result, key=lambda span: span.start)
    return result
    get_sort_key = lambda span: (span.end - span.start, -span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
     # Check for end - 1 here because boundaries are inclusive
     if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    result = sorted(result, key=lambda span: span.start)
    return result


def convert_to_spacy(data,path="/Users/fandi/Desktop/cibc_test/corups/train.spacy"):
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object
    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        
        for start, end, label in annot["entity"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        ents = filter_spans(ents)
        doc.set_ents(ents) # label the text with the ents
        db.add(doc)

    db.to_disk(path) # save the docbin object

# Set the keyword with label

In [44]:
dict_index_tr=[{'Statement of Work':'Contract_type','effective 01-May-17':"Start_date","For Compare Test - Org Confirmations":"Service type",
            'RR Agreement':'Parent_contract',"SN Agreement":"Parent_contract",
          "01-Jan-12":"Parent_Start_date"," 12];[Term & Renewal Notes: ]":"Renew",
            "ABC":"Internal Party","XYZ":"External Party"},{'Tenth Amendment':'Contract_type','effective 01-May-18':"Start_date",
            'Statement of Work':'Parent_contract',"for Compare Test effective": "Service type","RR Agreement":"Parent_contract",
          "31-Mar-16":"Parent_Start_date","01-Jan-12":"Parent_Start_date","SN Agreement":"Parent_contract","01-Jan-12":"Parent_Start_date",
            "ABC":"Internal Party","XYZ":"External Party"},{"Master Network Services Agreement":"Contract_type","effective 01-Oct-11":"Start_date",
                                                           "ABC":"Internal Party","XYZ (Canada) Ltd":"External Party",
                                                            "(in months): 6];[Term & Renewal Notes: ]": "Renew"}]
dict_index_t=[{'First Amendment':'Contract_type','effective 01-01-May-17':"Start_date","DOW": "Parent_contract",
            'For Compare Test - Org':'Service type',"RR Agreement":"Parent_contract",
          "12-Sep-16":"Parent_Start_date","RR Agreement":"Parent_contract","01-Jan-12":"Parent_Start_date",
            "SN Agreement":"Parent_contract","01-Jan-12":"Parent_Start_date",
            "ABC":"Internal Party","XYZ":"External Party"},{"Master Network Services Agreement":"Contract_type","effective 01-Oct-11":"Start_date",
                                                           "ABC":"Internal Party","XYZ (Canada) Ltd":"External Party",
                                                            "(in months): 6];[Term & Renewal Notes: ]": "Renew"}]

In [45]:
train=search_keyword_loc_old([df['description'][0],df['description'][3],df['description'][8]],dict_index_tr)


In [46]:
test=search_keyword_loc_old([df['description'][1],df['description'][8]],dict_index_t)

In [49]:
convert_to_spacy(data=train,path="/Users/fandi/Desktop/cibc_test/corups/train.spacy")
convert_to_spacy(data=test,path="/Users/fandi/Desktop/cibc_test/corups/dev.spacy")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [50]:
!python -m spacy init config --lang en --pipeline ner C:/Users/fandi/Desktop/deep_learning/config.cfg --force

[!] To generate a more effective transformer-based config (GPU-only), install
the spacy-transformers package and re-run this command. The config generated now
does not use transformers.
[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
C:\Users\fandi\Desktop\cibc_test\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2021-05-31 11:17:42.027605: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-05-31 11:17:42.027674: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Error processing line 7 of C:\Users\fandi\Anaconda3\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\fandi\Anaconda3\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


In [51]:
!python -m spacy train C:/Users/fandi/Desktop/deep_learning/config.cfg \
    --output C:/Users/fandi/Desktop/deep_learning/corups/  \
        --paths.train C:/Users/fandi/Desktop/deep_learning/corups/train.spacy \
            --paths.dev C:/Users/fandi/Desktop/deep_learning/corups/dev.spacy \
                --training.eval_frequency 15 --training.max_steps 150
#                
            

[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     28.50    0.00    0.00    0.00    0.00
  7      15          4.78    771.27    0.00    0.00    0.00    0.00
 15      30        314.59    617.47   26.09   33.33   21.43    0.26
 22      45          5.73    255.92   52.17   66.67   42.86    0.52
 30      60       1122.75    726.75   37.04   38.46   35.71    0.37
 37      75        285.52    413.11   50.00   60.00   42.86    0.50
 45      90       1693.20    305.44   75.86   73.33   78.57    0.76
 52     105          4.64     91.10   75.86   73.33   78.57    0.76
 60     120          2.53     35.57   74.07   76.92   71.43 

2021-05-31 11:17:56.562173: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-05-31 11:17:56.562294: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Error processing line 7 of C:\Users\fandi\Anaconda3\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\fandi\Anaconda3\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored
[2021-05-31 11:18:03,254] [INFO] Set up nlp object from config
[2021-05-31 11:18:03,272] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-05-31 11:18:03,278] [INFO] Created vocabulary
[2021-05-31 11:18:03,278] [INFO] Finished initializing nlp object
[2021-05-31 11:18:03,532] [INFO] Initialized pipeline componen

In [52]:
nlp1 = spacy.load(R"C:\Users\fandi\Desktop\deep_learning\corups\model-best") #Load the model

In [53]:
def test_model(data,model):
    doc_list=[]
    dict_res={""}
    for i in data:
        doc=model(i)
        doc_list.append(model(i))
        for ent in doc.ents:
            print(ent.text, ent.label_)
        spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter
        print("="*60)
    return doc_list

In [54]:
doc_list=test_model(df['description'],nlp1)

Statement of Work Contract_type
For Compare Test - Org Confirmations Service type
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


First Amendment Contract_type
effective 01-May-17 Start_date
DOW For Parent_contract
Compare Test - Org Confirmations Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party


Statement of Work Contract_type
for Compare Test effective Service type
31-Jun-27 Parent_Start_date
RR Agreement Parent_contract
01-Jan-26 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


Tenth Amendment Contract_type
effective 01-May-18 Start_date
Statement of Work Contract_type
for Compare Test effective Service type
31-Mar-16 Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party


Tenth Amendment Contract_type
effective 07-Jul-16 Start_date
Statement of Work Contract_type
for Compare Test effective Service type
31-Mar-16 Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party


DOW for BT Application Management Services Contract_type
effective 01-Dec-16 Start_date
effective 01-Jun-13 Start_date
ABC Internal Party
XYZ External Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


Parent = 100279] Contract_type
Second Amendment Contract_type
effective 01-Dec-19 Start_date
DOW Internal Party
for BCBT Application Management Services effective Service type
01-Dec-16 Parent_Start_date
effective 01-Jun-13 Start_date
ABC Internal Party
XYZ External Party
1 additional External Party
1 year] Renew


Third Amendment Contract_type
effective 01-Dec-20 Start_date
DOW Internal Party
for BCBT Application Management Services effective Service type
01-Dec-16 Parent_Start_date
effective 01-Jun-13 Start_date
ABC Internal Party
XYZ Inc.] External Party


Master Network Services Agreement Contract_type
effective 01-Oct-11 Start_date
ABC Internal Party
XYZ (Canada) External Party
(in months): 6];[Term & Renewal Notes: ] Renew


Statement of Work Contract_type
for Data Integrity Services effective Service type
03-Oct-16 Parent_Start_date
RR Agreement Parent_contract
01-Jan-12 Parent_Start_date
SN Agreement Parent_contract
ABC Internal Party
XYZ External Party
(in months): Renew
12];[Term & Renewal Notes: ] Renew


# Coreference Resolution
### 1 Neuralcoref
### 2 allennlp_models

In [3]:
from stanfordnlp.server import CoreNLPClient

In [4]:
import stanfordnlp
stanfordnlp.download('en')   # This downloads the English models for the neural pipeline


Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)


Default download directory: C:\Users\fandi\stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: C:\Users\fandi\stanfordnlp_resources\en_ewt_models.zip


100%|███████████████████████████████████████████████████████████████████████████████| 235M/235M [02:10<00:00, 1.81MB/s]



Download complete.  Models saved to: C:\Users\fandi\stanfordnlp_resources\en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


In [6]:
import stanza
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse') # This sets up a default neural pipeline in English


2021-06-10 00:34:10 INFO: Downloading default packages for language: en (English)...
2021-06-10 00:34:14 INFO: File exists: C:\Users\fandi\stanza_resources\en\default.zip.
2021-06-10 00:34:28 INFO: Finished downloading models and saved to C:\Users\fandi\stanza_resources.
2021-06-10 00:34:28 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-06-10 00:34:28 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2021-06-10 00:34:28 INFO: Use device: cpu
2021-06-10 00:34:28 INFO: Loading: tokenize
2021-06-10 00:34:28 INFO: Loading: pos
2021-06-10 00:34:29 INFO: Loading: lemma
2021-06-10 00:34:29 INFO: Loading: depparse
2021-06-10 00:34:30 INFO: Done loading processors!


In [17]:
from allennlp_models import pretrained

In [18]:
model=pretrained.load_predictor("coref-spanbert")

error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\fandi\Anaconda3\Anaconda3\envs\workspace\Lib\site-packages\allennlp_models\modelcards\lm-masked-language-model.json as plain json


In [20]:
df['description'][0]

'[Statement of Work For Compare Test - Org Confirmations effective 12-Sep-16 to RR Agreement effective 01-Jan-12 to SN Agreement effective 01-Jan-12 between ABC and XYZ, Inc];[Notice Timeframe: Prior to Expiration Date];[Renewal Interval (in months): 12];[Term & Renewal Notes: ]'

In [21]:
data=model.predict(df['description'][0])

In [22]:
from IPython.core.display import display, HTML
# Add css styles and js events to DOM, so that they are available to rendered html
import pandas as pd
display(HTML(open('visualization/highlight.css').read()))
display(HTML(open('visualization/highlight.js').read()))

import visualization

In [127]:
data3=model.predict(df_new['contract_overview'][6])
html = visualization.render(data3, allen=True, jupyter=False)
HTML(html)

In [130]:
df=pd.read_excel('data_masking.xlsx')

<ipython-input-130-02f3202cf741>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df=pd.read_excel('data_masking.xlsx')


In [132]:
data3=model.predict(df['overview'][0])
html = visualization.render(data3, allen=True, jupyter=False)
HTML(html)